In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
import pickle

In [2]:
pd.set_option('display.max_columns', None)

In [3]:

from sklearn.mixture import GaussianMixture

In [4]:
data1=pd.read_csv("kaggle_pivot_min_descending.csv")
data2=pd.read_csv("kaggle_pivot_raw.csv")
data3=pd.read_csv("kaggle_sw_errorfunc_descending_v01.csv")
data4=pd.read_csv("kaggle_sw_raw.csv")

In [5]:
branches = ['Civil Engineering', 'Computer Science and Engineering', 'Information Technology', 'Electrical Engineering', 'Electronics and Telecommunication Engg', 'Mechanical Engineering']
categories = ['NT 2 (NT-C)', 'SC', 'OBC', 'ST', 'OPEN']

filtered_data = data4[(data4["branch"].isin(branches)) & (data4["category"].isin(categories))]

columns_to_drop = ["fulfillment","rank","gender", "seat_type", "primary_seat_type", "secondary_seat_type", "branch_code", "enrollment_no"]
filtered_data = filtered_data.drop(columns=columns_to_drop)

filtered_data = filtered_data[filtered_data["score_type"] == "MHT-CET"]



In [6]:
filtered_data.tail(2)

,percentile,branch,category,score_type,college_name
104278,20.176312,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...
104279,12.935323,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...


In [7]:
max_percentile_rows = filtered_data.groupby(['college_name','branch', 'category']).apply(lambda x: x.loc[x['percentile'].idxmax()])

# Reset index to remove the groupby indices
max_percentile_rows.reset_index(drop=True, inplace=True)

In [8]:
filtered_data.college_name.nunique()

313

In [9]:
max_percentile_rows

,percentile,branch,category,score_type,college_name
0,59.063929,Civil Engineering,NT 2 (NT-C),MHT-CET,"A. P. Shah Institute of Technology, Thane"
1,63.088298,Civil Engineering,OBC,MHT-CET,"A. P. Shah Institute of Technology, Thane"
2,86.557640,Civil Engineering,OPEN,MHT-CET,"A. P. Shah Institute of Technology, Thane"
3,84.038206,Information Technology,NT 2 (NT-C),MHT-CET,"A. P. Shah Institute of Technology, Thane"
4,86.835611,Information Technology,OBC,MHT-CET,"A. P. Shah Institute of Technology, Thane"
...,...,...,...,...,...
3982,84.038206,Information Technology,SC,MHT-CET,Zeal Education Society's Zeal College of Engin...
3983,72.770241,Mechanical Engineering,NT 2 (NT-C),MHT-CET,Zeal Education Society's Zeal College of Engin...
3984,60.504324,Mechanical Engineering,OBC,MHT-CET,Zeal Education Society's Zeal College of Engin...
3985,69.168597,Mechanical Engineering,OPEN,MHT-CET,Zeal Education Society's Zeal College of Engin...


In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
#df=max_percentile_rows
df=filtered_data
columns_to_encode = ['college_name']
label_encoder = LabelEncoder()

for col in columns_to_encode:
    df[col + '_encoded'] = label_encoder.fit_transform(df[col])

#print("Original DataFrame:")
#print(df)


In [12]:
df.tail()

,percentile,branch,category,score_type,college_name,college_name_encoded
104275,63.547804,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...,245
104276,27.333743,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...,245
104277,20.232792,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...,245
104278,20.176312,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...,245
104279,12.935323,Electronics and Telecommunication Engg,OPEN,MHT-CET,Shree Siddheshwar Women's College Of Engineeri...,245


In [13]:
# decoded_columns = []

# for col in columns_to_encode:
#     encoded_col = col + '_encoded'
#     decoded_col = label_encoder.inverse_transform(df[encoded_col])
#     decoded_columns.append(decoded_col)

# decoded_df = pd.DataFrame(decoded_columns, index=columns_to_encode).T
# decoded_df.columns = [col.replace('_encoded', '') for col in decoded_df.columns]

# # Combine the decoded columns with the original DataFrame
# decoded_df = pd.concat([df.drop(columns=[col + '_encoded' for col in columns_to_encode]), decoded_df], axis=1)

# # print("Decoded DataFrame:")
# # print(decoded_df)


In [14]:
############Decoded to Encode




# decoded_values = label_encoder.inverse_transform(df['college_name_encoded'])

# # Add the decoded values back to the DataFrame
# df['college_name_decoded'] = decoded_values

# Print the DataFrame with the decoded values
#print(df)

In [15]:
# a= label_encoder.inverse_transform([2])
# a

In [16]:
df_encoded = pd.get_dummies(df, columns=['branch', 'category'])


In [17]:
df_encoded.head()

,percentile,score_type,college_name,college_name_encoded,branch_Civil Engineering,branch_Computer Science and Engineering,branch_Electrical Engineering,branch_Electronics and Telecommunication Engg,branch_Information Technology,branch_Mechanical Engineering,category_NT 2 (NT-C),category_OBC,category_OPEN,category_SC,category_ST
0,90.121473,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,1,0,0,0,0
1,89.889223,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,0,0,0,1,0
2,89.540152,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,0,1,0,0,0
3,88.241971,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,0,1,0,0,0
5,87.898633,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,0,1,0,0,0


In [18]:
# def predict_college(percentile, branch, category):
#     # Create a DataFrame with the input values
#     input_data = pd.DataFrame({
#         'percentile': [percentile],
#         'branch_Civil Engineering': [0],
#         'branch_Computer Science and Engineering': [0],
#         'branch_Electrical Engineering': [0],
#         'branch_Electronics and Telecommunication Engg': [0],
#         'branch_Information Technology': [0],
#         'branch_Mechanical Engineering': [0],
#         'category_NT 2 (NT-C)': [0],
#         'category_OBC': [0],
#         'category_OPEN': [0],
#         'category_SC': [0],
#         'category_ST': [0]
#     })
    
#     # Set the corresponding branch and category columns to 1 based on user input
#     input_data['branch_' + branch] = 1
#     input_data['category_' + category] = 1
    
#     # Predict using the trained model
#     college_encoded = clf.predict(input_data)[0]
    
#     # Retrieve the college name from the encoded value
#     college_name = df[df['college_name_encoded'] == college_encoded]['college_name'].iloc[0]
    
#     return college_name

# # Example usage:
# percentile = float(input("Enter percentile: "))
# branch = input("Enter branch (Civil Engineering, Computer Science and Engineering, Electrical Engineering, Electronics and Telecommunication Engg, Information Technology, Mechanical Engineering): ")
# category = input("Enter category (NT 2 (NT-C), OBC, OPEN, SC, ST): ")

# predicted_college = predict_college(percentile, branch, category)
# print("Predicted College:", predicted_college)


In [19]:
# with open("model.pkl", "wb") as f:
#     pickle.dump(clf, f)

In [20]:
# decoded_columns = []

# for col in columns_to_encode:
#     encoded_col = col + '_encoded'
#     decoded_col = label_encoder.inverse_transform(df[encoded_col])
#     decoded_columns.append(decoded_col)

# decoded_df = pd.DataFrame(decoded_columns, index=columns_to_encode).T
# decoded_df.columns = [col.replace('_encoded', '') for col in decoded_df.columns]

# # Combine the decoded columns with the original DataFrame
# decoded_df = pd.concat([df.drop(columns=[col + '_encoded' for col in columns_to_encode]), decoded_df], axis=1)

# print("Decoded DataFrame:")
# print(decoded_df)


In [21]:
df_encoded.head(2)

,percentile,score_type,college_name,college_name_encoded,branch_Civil Engineering,branch_Computer Science and Engineering,branch_Electrical Engineering,branch_Electronics and Telecommunication Engg,branch_Information Technology,branch_Mechanical Engineering,category_NT 2 (NT-C),category_OBC,category_OPEN,category_SC,category_ST
0,90.121473,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,1,0,0,0,0
1,89.889223,MHT-CET,"Government College of Engineering, Amravati",79,1,0,0,0,0,0,0,0,0,1,0


In [24]:
df=df_encoded
X = df.drop(columns=['score_type','college_name','college_name_encoded'])
y = df['college_name_encoded']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize the XGBoost Classifier
clf = xgb.XGBClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.08782225948438482


In [ ]:
ffffffff

In [27]:
from sklearn.ensemble import RandomForestClassifier
df=df_encoded
X = df.drop(columns=['score_type','college_name','college_name_encoded'])
y = df['college_name_encoded']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize the Random Forest Classifier
clf_rf = RandomForestClassifier()

# Train the classifier
clf_rf.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = clf_rf.predict(X_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Classifier Accuracy:", accuracy_rf)

Random Forest Classifier Accuracy: 0.058478306434709706


In [ ]:
ffffffffff

In [31]:
# import pandas as pd
# import time
# import pickle
# from sklearn.preprocessing import LabelEncoder

# # Load the dataset
# data4 = pd.read_csv("kaggle_sw_raw.csv")

# columns_to_encode = ['college_name']
# label_encoder = LabelEncoder()

# for col in columns_to_encode:
#     data4[col + '_encoded'] = label_encoder.fit_transform(data4[col])

# # Define branches and categories
# branches = ['Civil Engineering', 'Computer Science and Engineering', 'Information Technology', 'Electrical Engineering', 'Electronics and Telecommunication Engg', 'Mechanical Engineering']
# categories = ['NT 2 (NT-C)', 'SC', 'OBC', 'ST', 'OPEN']

# # Filter the data based on branches and categories
# filtered_data = data4[(data4["branch"].isin(branches)) & (data4["category"].isin(categories))]

# # Drop irrelevant columns
# columns_to_drop = ["fulfillment", "rank", "gender", "seat_type", "primary_seat_type", "secondary_seat_type", "branch_code", "enrollment_no"]
# filtered_data = filtered_data.drop(columns=columns_to_drop)

# # Filter by score type
# filtered_data = filtered_data[filtered_data["score_type"] == "MHT-CET"]

# # Load the trained model
# with open("model.pkl", "rb") as f:
#     clf = pickle.load(f)

# # Prediction function
# def predict_college(percentile, branch, category):
#     # Create DataFrame for prediction
#     input_data = pd.DataFrame({
#         'percentile': [percentile],
#         'branch_encoded': [0],
#         'category_encoded': [0]
#     })

#     # Set the corresponding branch and category columns to 1 based on user input
#     input_data.loc[0, 'branch_encoded'] = label_encoder.transform([branch])[0]
#     input_data.loc[0, 'category_encoded'] = label_encoder.transform([category])[0]
    
#     # Predict using the trained model
#     college_encoded = clf.predict(input_data)[0]
#     college_name = filtered_data[filtered_data['college_name_encoded'] == college_encoded]['college_name'].iloc[0]
    
#     return college_name

# # UI
# print("Predict Preferential allotment of college for admission 🎓")

# # Add input widgets
# percentage = float(input('Enter a MHT-CET Score (Percentage): '))
# gender = input("Select Gender (Male/Female): ")
# branch = input("Select Branch: " + str(filtered_data["branch"].unique()) + ": ")
# category = input("Select Reservation Category: " + str(filtered_data["category"].unique()) + ": ")

# # Multiselect preferred colleges
# selected_colleges = input('Select preferred colleges (comma-separated): ').split(',')

# print('Selected colleges:')
# for college in selected_colleges:
#     print(college)

# # Prediction
# print('Predicting...')
# predicted_college = predict_college(percentage, branch, category)
# time.sleep(2)  # Simulate a delay
# print(f'Predicted College: {predicted_college}')

# # Check if predicted college is in selected colleges
# if predicted_college in selected_colleges:
#     print("Congratulations! You have selected the predicted college.")
# else:
#     print("Sorry, the predicted college is not in your selected colleges.")


In [28]:
# with open("model.pkl", "wb") as f:
#     pickle.dump(clf, f)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score

df=df_encoded
X = df.drop(columns=['score_type','college_name','college_name_encoded'])
y = df['college_name_encoded']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter Tuning
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# Initialize the XGBoost Classifier
clf = xgb.XGBClassifier()

# Grid Search
grid_search = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Train the classifier with the best parameters
best_clf = xgb.XGBClassifier(**best_params)
best_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = best_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Assuming df contains your dataset

# Feature Engineering
# Assuming you've already prepared your features X and target variable y

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost Classifier
clf = XGBClassifier()

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [5, 10, 15],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Train the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Predict on the test set using the best model
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
# import pickle

# with open("model.pkl", "wb") as f:
# pickle.dump(model, f)

In [ ]:
#predicted_encoded_college=1

In [ ]:
#predicted_college_name = label_encoder['college_name'].inverse_transform([predicted_encoded_college])

In [ ]:
# # Decode predicted college name
# predicted_college_name = label_encoder['college_name'].inverse_transform([predicted_encoded_college])[0]

# print("Predicted College Name:", predicted_college_name)
